In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU, Dropout

from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

In [ ]:
!pip install BnPreprocessing


In [ ]:
#read Data
data =pd.read_excel('/content/drive/MyDrive/Thesis/Shared/DFBA Train Data.xlsx')[0:6475]
data_df = data
text_data = data_df["text"]
labels = data_df["label2"]
classes = data_df["class2"]



In [ ]:
set(labels)

In [ ]:
def make_four_label_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "mild"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "moderate"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "severe"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement' ):
        tag = "no"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  return new_df
new_df = make_four_label_droping(data_df)

In [ ]:
def make_two_class(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if data_df.label2[i] == 'no': 
        tag = "non-depressed"
        tag_list.append(tag)
      else: 
        tag = "depressed"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df
new_df = make_two_class(data_df)

In [ ]:
def make_two_tag_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement' ):
        tag = "non-depressive"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  

  return new_df
new_df = make_two_tag_droping(data_df)

In [ ]:
text_data = new_df["text"]
labels = new_df["label"]

In [ ]:
set(labels)

In [ ]:
!pip install BnPreprocessing

In [ ]:
import BnPreprocessing as pp
text_data = text_data.apply(pp.remove_noise)

In [ ]:
new_df.to_excel("/content/drive/MyDrive/Thesis/Thesis Code/BenFEDAS_two_label.xlsx", index = False)

In [ ]:

#Encoding
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
labels = label.fit_transform(labels)

In [ ]:
#Encoding
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
labels = label.fit_transform(labels)

In [ ]:
#One hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)

class_labels = labels.values.reshape((labels.shape[0], 1))
labels = encoder.fit_transform(class_labels)


In [ ]:
import BnPreprocessing as pp
text_data = text_data.apply(pp.remove_noise)

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2, shuffle = True, random_state=42)
#train_data, sp_data, train_labels, sp_labels = train_test_split(text_data, labels, test_size=0.4, shuffle = False, random_state=42)
#val_data, test_data, val_labels, test_labels = train_test_split(sp_data, sp_labels, test_size=0.5, shuffle = False, random_state=42)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(text_data)
vocab_size = len(tokenizer.word_index) + 1
max_length = 196
embedding_dim = 300
trunc_type='post'
padding_type='post'
 
training_sequences = tokenizer.texts_to_sequences(train_data)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#val_sequences = tokenizer.texts_to_sequences(val_data)
#val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(train_labels)
#val_padded = np.array(val_padded)
#val_labels = np.array(val_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(test_labels)

In [ ]:

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim=vocab_size, 
                              output_dim=embedding_dim, 
                              input_length=max_length),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(4, activation ='softmax'),
  ])
  

#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
tensorboard_callback  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
num_epochs = 50
history = model.fit(training_padded, training_labels,epochs=num_epochs, batch_size =32, validation_data=(testing_padded, testing_labels), validation_batch_size =16, callbacks=[tensorboard_callback], verbose=2)

In [ ]:
lr =tf.keras.optimizers.Adam(
    learning_rate= 1e-04,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    amsgrad=False,
    name="Adam"
)
model =  tf.keras.Sequential([
                              tf.keras.layers.Embedding(input_dim=vocab_size, 
                              output_dim=embedding_dim, 
                              input_length=max_length),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                              tf.keras.layers.Dropout(0.8),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                              tf.keras.layers.Dropout(0.8),
                              tf.keras.layers.Dense(32,kernel_regularizer=regularizers.l2(0.01),activation="relu"),
                              tf.keras.layers.Dropout(0.2),
                              tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=lr,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer=lr,metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
tensorboard_callback  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
num_epochs = 50
history = model.fit(training_padded, training_labels,epochs=num_epochs, batch_size =32, validation_data=(testing_padded, testing_labels), validation_batch_size =16, callbacks=[tensorboard_callback], verbose=2)

In [ ]:
model.summary()

In [ ]:
## অ্যাক্যুরেসি  ম্যাটপ্লটলিব দিয়ে

import matplotlib.pyplot as plt

def plot_graphs(history, string):
  
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.savefig("/content/drive/MyDrive/Thesis/Thesis Code/thesis result/196l_200_64_32_unit_Accuracy .2_test_.8.8.2dropout_4bilstm drop.eps",format = 'eps',dpi = 1000)
  
plot_graphs(history, "accuracy")


In [ ]:
##  লস, ম্যাটপ্লটলিব দিয়ে

import matplotlib.pyplot as plt

def plot_graphs(history, string):
  
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.savefig("/content/drive/MyDrive/Thesis/Thesis Code/thesis result/196l_200_64_32_unit_loss .2_test_.8.8.2dropout_4bilstm drop.eps",format = 'eps',dpi = 1000)
  
plot_graphs(history, "loss")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
ypred = model.predict(testing_padded, verbose=2)
ypred = np.argmax(ypred, axis=1)
print(classification_report(test_labels, ypred))
confusion_matrix(test_labels, ypred)


In [ ]:

model.evaluate(test_data, test_labels, verbose=0)


In [ ]:
test_labels.shape

In [ ]:
ypred.shape

In [ ]:
import pandas
report = classification_report(test_labels, ypred, output_dict=True)
df = pandas.DataFrame(report).transpose().round(decimals=2)
df.to_csv('/content/drive/MyDrive/Thesis/Result/effect_length_embedding/200l_300e__20_epoch_report_4_test_bilstm_drop.csv')
report1 = confusion_matrix(test_labels, ypred)
df = pandas.DataFrame(report1)
df.to_csv('/content/drive/MyDrive/Thesis/Result/effect_length_embedding/200l_300e_20_epoch_confusion_4_test_bilstm drop.csv')